<a href="https://colab.research.google.com/github/alexlauforgithub/mystuffs/blob/master/%E8%B2%A1%E9%87%91%E8%B3%87%E8%A8%8A%E7%88%AC%E8%9F%B2%E6%A0%B8%E5%BF%83.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fake_useragent

  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-py3-none-any.whl size=13503 sha256=c032a127a4826664403385cba147a219a89be52a903adae3bce01e64cfbce74d
  Stored in directory: /root/.cache/pip/wheels/ed/f7/62/50ab6c9a0b5567267ab76a9daa9d06315704209b2c5d032031
Successfully built fake-useragent


In [ ]:
import pandas as pd
import numpy as np
import requests
import io
import os
import time
import datetime
from fake_useragent import UserAgent
# import mysql.connector

In [ ]:
# os.chdir("E:/tw_stock")

## 爬取大盤歷史資料

In [ ]:
# 日期設定
first_date = datetime.datetime.today().replace(day=1).strftime('%Y-%m-%d')
start_date = '2000-01-01'
date_list = pd.date_range(start_date, first_date, freq='MS')
date_list = date_list.strftime('%Y%m%d').tolist()

user_agent = UserAgent(use_cache_server=False)
user_agent = UserAgent()
for i in date_list:
    print("目前進度 : ", date_list.index(i), "/", len(date_list), "\n")
    url = 'https://www.twse.com.tw/indicesReport/MI_5MINS_HIST?response=csv&date='+ i 
    res = requests.get(url, headers={ 'user-agent': user_agent.random })
    text = res.text
    
    if len(text) == 0:
        print(i +" is holiday!")
        time.sleep(10)
    else:
        # 把爬下來的資料整理乾淨
        cleaned_data = []
        for da in text.split('\n'):
            cleaned_data.append([ele.replace('",\r','').replace('"','') 
                                for ele in da.split('","')])
        
        # 輸出成表格並呈現在excel上
        df = pd.DataFrame(cleaned_data, columns = cleaned_data[1])[2:].dropna().reset_index(drop=True)
        df.columns = ['date', 'open', 'high', 'low', 'close']
        #民國年轉西元年
        for ix in range(len(df['date'])):
            df['date'].iloc[ix]=df['date'].iloc[ix].replace(df['date'].iloc[ix][0:3], str(int(df['date'].iloc[ix][0:3]) + 1911))
        # 轉成數值
        df['date'] = df['date'].str.replace('/', '').astype(int)
        
        if i == date_list[0]:
            data = df
        else:
            data = data.append(df).reset_index(drop=True)
            time.sleep(10)
data.to_csv('data/tw_index.csv', sep=',', encoding='utf-8', index=False)
print("Saving Complete!")

IndexError: ignored

## 爬取股價歷史資料

In [ ]:
# 日期設定
today_date = datetime.datetime.today().strftime('%Y-%m-%d')
start_date = '2004-02-11' # 證交所最早提供的日期
date_list = pd.date_range(start=start_date ,end=today_date).strftime('%Y%m%d').tolist()

In [ ]:
# 爬取證交所股價資料
user_agent = UserAgent(use_cache_server=False)
user_agent = UserAgent()
for i in date_list:
    print("目前進度 : ", date_list.index(i), "/", (len(date_list)-1), "\n")
    url = 'https://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date='+ i + '&type=ALLBUT0999'
    res = requests.get(url, headers={ 'user-agent': user_agent.random })
    text = res.text
    
    if len(text) == 0:
        print(i +" is holiday!")
    else:
        # 把爬下來的資料整理乾淨
        cleaned_data = []
        for da in text.split('\n'):
            if len(da.split('","')) == 16 and da.split('","')[0][0] != '=':
                cleaned_data.append([ele.replace('",\r','').replace('"','') 
                                     for ele in da.split('","')])
        
        # 輸出成表格並呈現在excel上
        df = pd.DataFrame(cleaned_data, columns = cleaned_data[0])[1:]
        df['日期'] = i
        df = df[['日期', '證券代號', '證券名稱', '開盤價', '最高價', '最低價', '收盤價', '成交股數', '成交金額']]
        df.columns = ['date', 'code', 'name', 'open', 'high', 'low', 'close', 'volume_stocks', 'value']
        df = df[df['code'].apply(lambda x: len(x) < 5)].reset_index(drop=True)
        
        if i == date_list[0]:
            data = df
        else:
            data = data.append(df).reset_index(drop=True)
            time.sleep(10)
            
# data.to_csv('data/ETF_Data.csv', sep=',', encoding='utf-8', index=False)
# print("Saving Complete!")

In [ ]:
data.to_csv('data/tw_stock4.csv', sep=',', encoding='utf-8', index=False)
print("Saving Complete!")

## 每日資料更新

In [ ]:
#爬取當日大盤資料
today_date = datetime.datetime.today().replace(day=1).strftime('%Y%m%d')
right_date = datetime.datetime.today().strftime('%Y%m%d')
user_agent = UserAgent(use_cache_server=False)
user_agent = UserAgent()
url = 'https://www.twse.com.tw/indicesReport/MI_5MINS_HIST?response=csv&date=' + today_date
res = requests.get(url, headers={ 'user-agent': user_agent.random })
text = res.text
cleaned_data = []
for da in text.split('\n'):
    cleaned_data.append([ele.replace('",\r','').replace('"','') 
                        for ele in da.split('","')])

df = pd.DataFrame(cleaned_data, columns = cleaned_data[1])[2:].dropna().reset_index(drop=True)
df.columns = ['date', 'open', 'high', 'low', 'close']
for i in range(len(df['date'])):
    df['date'].iloc[i]=df['date'].iloc[i].replace(df['date'].iloc[i][0:3], str(int(df['date'].iloc[i][0:3]) + 1911))
df['date'] = df['date'].str.replace('/', '').astype(int)
df.to_csv('data/tw_index_' + right_date + '.csv', sep=',', encoding='utf-8', index=False)
print("Saving Complete!")

Saving Complete!


In [ ]:
df

,date,code,name,open,high,low,close,volume_stocks,value
0,20210803,1101,台泥,53.10,53.90,53.00,53.90,"45,545,154","2,441,886,377"
1,20210803,1102,亞泥,52.00,52.20,51.70,52.10,"26,329,816","1,368,005,256"
2,20210803,1103,嘉泥,22.45,22.60,22.35,22.40,"1,349,450","30,266,210"
3,20210803,1104,環泥,23.00,23.05,22.90,23.00,"491,062","11,280,176"
4,20210803,1108,幸福,12.90,13.10,12.85,12.95,"332,987","4,313,229"
...,...,...,...,...,...,...,...,...,...
950,20210803,9944,新麗,22.40,22.60,22.20,22.60,"134,437","3,011,803"
951,20210803,9945,潤泰新,63.90,64.30,63.30,64.30,"3,757,618","239,676,808"
952,20210803,9946,三發地產,15.20,15.55,15.20,15.55,"290,830","4,456,465"
953,20210803,9955,佳龍,18.25,18.45,18.10,18.30,"78,682","1,435,593"


In [ ]:
text

'"110年08月03日 價格指數(臺灣證券交易所)"\r\n"指數","收盤指數","漲跌(+/-)","漲跌點數","漲跌百分比(%)","特殊處理註記",\r\n"寶島股價指數","20,046.06","+","60.69","0.30","",\r\n"發行量加權股價指數","17,553.76","+","50.48","0.29","",\r\n"臺灣公司治理100指數","10,063.86","+","47.46","0.47","",\r\n"臺灣50指數","13,731.90","+","51.78","0.38","",\r\n"臺灣50權重上限30%指數","12,985.37","+","35.24","0.27","",\r\n"臺灣中型100指數","14,108.80","+","35.15","0.25","",\r\n"臺灣資訊科技指數","23,736.40","+","124.57","0.53","",\r\n"臺灣發達指數","11,913.75","+","15.42","0.13","",\r\n"臺灣高股息指數","7,676.69","-","3.95","-0.05","",\r\n"臺灣就業99指數","10,036.57","+","20.69","0.21","",\r\n"臺灣高薪100指數","9,086.17","-","5.23","-0.06","",\r\n"未含金融指數","15,403.70","+","52.53","0.34","",\r\n"未含電子指數","19,791.48","-","20.42","-0.10","",\r\n"未含金融電子指數","17,346.39","-","13.58","-0.08","",\r\n"小型股300指數","9,464.60","+","10.70","0.11","",\r\n"水泥窯製類指數","844.10","+","8.36","1.00","",\r\n"塑膠化工類指數","969.75","-","4.26","-0.44","",\r\n"機電類指數","4,649.90","+","25.05","0.54","",\r\n"水泥類指數","208.90","+","2.71","1.31","",\r\n"食品類指

In [ ]:
# 爬取每日股價資料
today_date = '20210723'
today_date = datetime.datetime.today().strftime('%Y%m%d')
user_agent = UserAgent()

url = 'https://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date='+ today_date + '&type=ALLBUT0999'
res = requests.get(url, headers={ 'user-agent': user_agent.random })
text = res.text
    
# 把爬下來的資料整理乾淨
cleaned_data = []
for da in text.split('\n'):
    if len(da.split('","')) == 16 and da.split('","')[0][0] != '=':
        cleaned_data.append([ele.replace('",\r','').replace('"','') 
                            for ele in da.split('","')])
        
# 輸出成表格並呈現在excel上
df = pd.DataFrame(cleaned_data, columns = cleaned_data[0])[1:]
df['日期'] = today_date
df = df[['日期', '證券代號', '證券名稱', '開盤價', '最高價', '最低價', '收盤價', '成交股數', '成交金額']]
df.columns = ['date', 'code', 'name', 'open', 'high', 'low', 'close', 'volume_stocks', 'value']
df = df[df['code'].apply(lambda x: len(x) < 5)].reset_index(drop=True)
df.to_csv('data/tw_stock_' + today_date + '.csv', sep=',', encoding='utf-8', index=False)
print("Saving Complete!")

Error occurred during loading data. Trying to use cache server https://fake-useragent.herokuapp.com/browsers/0.1.11
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/fake_useragent/utils.py", line 154, in load
    for item in get_browsers(verify_ssl=verify_ssl):
  File "/usr/local/lib/python3.7/dist-packages/fake_useragent/utils.py", line 99, in get_browsers
    html = html.split('<table class="w3-table-all notranslate">')[1]
IndexError: list index out of range


FileNotFoundError: ignored

## 爬取證交所籌碼資料

In [ ]:
# 日期設定
today_date = datetime.datetime.today().strftime('%Y-%m-%d')
start_date = '2014-12-01' # 證交所最早提供的日期
date_list = pd.date_range(start=start_date ,end=today_date).strftime('%Y%m%d').tolist()

In [ ]:
# 爬取證交所籌碼資料

'https://www.twse.com.tw/fund/T86?response=csv&date=20210802&selectType=ALLBUT0999'

user_agent = UserAgent(use_cache_server=False)
user_agent = UserAgent()
for i in date_list:
    print("目前進度 : ", date_list.index(i), "/", (len(date_list)-1), "\n")
    url = 'https://www.twse.com.tw/fund/T86?response=csv&date='+ i + '&selectType=ALLBUT0999'
    res = requests.get(url, headers={ 'user-agent': user_agent.random })
    text = res.text
    ix = int(i)
    
    if len(text) == 0 or text == '\r\n':
        print(i +" is holiday!")
        time.sleep(11)
    elif ix < 20141201:
        # 把爬下來的資料整理乾淨
        cleaned_data = []
        for da in text.split('\n'):
            if len(da.split('","')) == 12 and da.split('","')[0][0] != '=':
                cleaned_data.append([ele.replace('",\r','').replace('"','') 
                                     for ele in da.split('","')])
        
        # 輸出成表格並呈現在excel上
        df = pd.DataFrame(cleaned_data, columns = cleaned_data[0])[1:]
        df.columns = ['code', 'name', 'FI_BUY', 'FI_SELL', 'FI_TOTAL', 'SIT_BUY', 'SIT_SELL', 'SIT_TOTAL', 'DL_TOTAL', 'DL_BUY', 'DL_SELL', 'TOP3_TOTAL']
        df['date'] = i
        df = df[['date', 'code', 'name', 'FI_BUY', 'FI_SELL', 'FI_TOTAL', 'SIT_BUY', 'SIT_SELL', 'SIT_TOTAL', 'DL_TOTAL', 'DL_BUY', 'DL_SELL', 'TOP3_TOTAL']]
        df = df[df['code'].apply(lambda x: len(x) < 5)].reset_index(drop=True)
        
        
        if i == date_list[0]:
            data = df
            time.sleep(11)
        else:
            data = data.append(df).reset_index(drop=True)
            time.sleep(11)
        
        if ix == 20141128:
            data.to_csv('data/top3_total_1.csv', sep=',', encoding='utf-8', index=False)
            print("Saving Complete!")
        
    elif ix < 20171218 and ix >= 20141201 :
        # 把爬下來的資料整理乾淨
        cleaned_data = []
        for da in text.split('\n'):
            if len(da.split('","')) == 16 and da.split('","')[0][0] != '=':
                cleaned_data.append([ele.replace('",\r','').replace('"','') 
                                     for ele in da.split('","')])
        
        # 輸出成表格並呈現在excel上
        df = pd.DataFrame(cleaned_data, columns = cleaned_data[0])[1:]
        df.columns = ['code', 'name', 'FI_BUY', 'FI_SELL', 'FI_TOTAL', 'SIT_BUY', 'SIT_SELL', 'SIT_TOTAL', 'DL_TOTAL', 'DL_SELF_BUY', 'DL_SELF_SELL', 'DL_SELF_TOTAL', 'DL_HEDGE_BUY', 'DL_HEDGE_SELL', 'DL_HEDGE_TOTAL', 'TOP3_TOTAL']
        df['date'] = i
        df = df[['date', 'code', 'name', 'FI_BUY', 'FI_SELL', 'FI_TOTAL', 'SIT_BUY', 'SIT_SELL', 'SIT_TOTAL', 'DL_TOTAL', 'DL_SELF_BUY', 'DL_SELF_SELL', 'DL_SELF_TOTAL', 'DL_HEDGE_BUY', 'DL_HEDGE_SELL', 'DL_HEDGE_TOTAL', 'TOP3_TOTAL']]
        df = df[df['code'].apply(lambda x: len(x) < 5)].reset_index(drop=True)
        
        if ix == 20141201:
            data = df
            time.sleep(11)
        else:
            data = data.append(df).reset_index(drop=True)
            time.sleep(11)
            
        if ix == 20171215:
            data.to_csv('data/top3_total_2.csv', sep=',', encoding='utf-8', index=False)
            print("Saving Complete!")
            
    elif ix >= 20171218:
        # 把爬下來的資料整理乾淨
        cleaned_data = []
        for da in text.split('\n'):
            if len(da.split('","')) == 19 and da.split('","')[0][0] != '=':
                cleaned_data.append([ele.replace('",\r','').replace('"','') 
                                     for ele in da.split('","')])
        
        # 輸出成表格並呈現在excel上
        df = pd.DataFrame(cleaned_data, columns = cleaned_data[0])[1:]
        df.columns = ['code', 'name', 'FI_BUY', 'FI_SELL', 'FI_TOTAL', 'FI_SELF_BUY', 'FI_SELF_SELL', 'FI_SELF_TOTAL', 'SIT_BUY', 'SIT_SELL', 'SIT_TOTAL', 'DL_TOTAL', 'DL_SELF_BUY', 'DL_SELF_SELL', 'DL_SELF_TOTAL', 'DL_HEDGE_BUY', 'DL_HEDGE_SELL', 'DL_HEDGE_TOTAL', 'TOP3_TOTAL']
        df['date'] = i
        df = df[['date', 'code', 'name', 'FI_BUY', 'FI_SELL', 'FI_TOTAL', 'FI_SELF_BUY', 'FI_SELF_SELL', 'FI_SELF_TOTAL', 'SIT_BUY', 'SIT_SELL', 'SIT_TOTAL', 'DL_TOTAL', 'DL_SELF_BUY', 'DL_SELF_SELL', 'DL_SELF_TOTAL', 'DL_HEDGE_BUY', 'DL_HEDGE_SELL', 'DL_HEDGE_TOTAL', 'TOP3_TOTAL']]
        df = df[df['code'].apply(lambda x: len(x) < 5)].reset_index(drop=True)
        
        if ix == 20171218:
            data = df
            time.sleep(11)
        else:
            data = data.append(df).reset_index(drop=True)
            time.sleep(11)
data.to_csv('data/top3_total_3.csv', sep=',', encoding='utf-8', index=False)
print("Saving Complete!")

目前進度 :  0 / 2413 

目前進度 :  1 / 2413 

目前進度 :  2 / 2413 

目前進度 :  3 / 2413 

目前進度 :  4 / 2413 

目前進度 :  5 / 2413 

20141206 is holiday!
目前進度 :  6 / 2413 

20141207 is holiday!
目前進度 :  7 / 2413 

目前進度 :  8 / 2413 

目前進度 :  9 / 2413 

目前進度 :  10 / 2413 

目前進度 :  11 / 2413 

目前進度 :  12 / 2413 

20141213 is holiday!
目前進度 :  13 / 2413 

20141214 is holiday!
目前進度 :  14 / 2413 

目前進度 :  15 / 2413 

目前進度 :  16 / 2413 

目前進度 :  17 / 2413 

目前進度 :  18 / 2413 

目前進度 :  19 / 2413 

20141220 is holiday!
目前進度 :  20 / 2413 

20141221 is holiday!
目前進度 :  21 / 2413 

目前進度 :  22 / 2413 

目前進度 :  23 / 2413 

目前進度 :  24 / 2413 

目前進度 :  25 / 2413 

目前進度 :  26 / 2413 

目前進度 :  27 / 2413 

20141228 is holiday!
目前進度 :  28 / 2413 

目前進度 :  29 / 2413 

目前進度 :  30 / 2413 

目前進度 :  31 / 2413 

20150101 is holiday!
目前進度 :  32 / 2413 

20150102 is holiday!
目前進度 :  33 / 2413 

20150103 is holiday!
目前進度 :  34 / 2413 

20150104 is holiday!
目前進度 :  35 / 2413 

目前進度 :  36 / 2413 

目前進度 :  37 / 2413 

目前進度 :  38 / 2413 


20150926 is holiday!
目前進度 :  300 / 2413 

20150927 is holiday!
目前進度 :  301 / 2413 

20150928 is holiday!
目前進度 :  302 / 2413 

20150929 is holiday!
目前進度 :  303 / 2413 

目前進度 :  304 / 2413 

目前進度 :  305 / 2413 

目前進度 :  306 / 2413 

20151003 is holiday!
目前進度 :  307 / 2413 

20151004 is holiday!
目前進度 :  308 / 2413 

目前進度 :  309 / 2413 

目前進度 :  310 / 2413 

目前進度 :  311 / 2413 

目前進度 :  312 / 2413 

20151009 is holiday!
目前進度 :  313 / 2413 

20151010 is holiday!
目前進度 :  314 / 2413 

20151011 is holiday!
目前進度 :  315 / 2413 

目前進度 :  316 / 2413 

目前進度 :  317 / 2413 

目前進度 :  318 / 2413 

目前進度 :  319 / 2413 

目前進度 :  320 / 2413 

20151017 is holiday!
目前進度 :  321 / 2413 

20151018 is holiday!
目前進度 :  322 / 2413 

目前進度 :  323 / 2413 

目前進度 :  324 / 2413 

目前進度 :  325 / 2413 

目前進度 :  326 / 2413 

目前進度 :  327 / 2413 

20151024 is holiday!
目前進度 :  328 / 2413 

20151025 is holiday!
目前進度 :  329 / 2413 

目前進度 :  330 / 2413 

目前進度 :  331 / 2413 

目前進度 :  332 / 2413 

目前進度 :  333 / 2413 

目前進度 :  334 /

目前進度 :  591 / 2413 

目前進度 :  592 / 2413 

目前進度 :  593 / 2413 

20160716 is holiday!
目前進度 :  594 / 2413 

20160717 is holiday!
目前進度 :  595 / 2413 

目前進度 :  596 / 2413 

目前進度 :  597 / 2413 

目前進度 :  598 / 2413 

目前進度 :  599 / 2413 

目前進度 :  600 / 2413 

20160723 is holiday!
目前進度 :  601 / 2413 

20160724 is holiday!
目前進度 :  602 / 2413 

目前進度 :  603 / 2413 

目前進度 :  604 / 2413 

目前進度 :  605 / 2413 

目前進度 :  606 / 2413 

目前進度 :  607 / 2413 

20160730 is holiday!
目前進度 :  608 / 2413 

20160731 is holiday!
目前進度 :  609 / 2413 

目前進度 :  610 / 2413 

目前進度 :  611 / 2413 

目前進度 :  612 / 2413 

目前進度 :  613 / 2413 

目前進度 :  614 / 2413 

20160806 is holiday!
目前進度 :  615 / 2413 

20160807 is holiday!
目前進度 :  616 / 2413 

目前進度 :  617 / 2413 

目前進度 :  618 / 2413 

目前進度 :  619 / 2413 

目前進度 :  620 / 2413 

目前進度 :  621 / 2413 

20160813 is holiday!
目前進度 :  622 / 2413 

20160814 is holiday!
目前進度 :  623 / 2413 

目前進度 :  624 / 2413 

目前進度 :  625 / 2413 

目前進度 :  626 / 2413 

目前進度 :  627 / 2413 

目前進度 :  628 /

目前進度 :  883 / 2413 

目前進度 :  884 / 2413 

目前進度 :  885 / 2413 

目前進度 :  886 / 2413 

目前進度 :  887 / 2413 

20170506 is holiday!
目前進度 :  888 / 2413 

20170507 is holiday!
目前進度 :  889 / 2413 

目前進度 :  890 / 2413 

目前進度 :  891 / 2413 

目前進度 :  892 / 2413 

目前進度 :  893 / 2413 

目前進度 :  894 / 2413 

20170513 is holiday!
目前進度 :  895 / 2413 

20170514 is holiday!
目前進度 :  896 / 2413 

目前進度 :  897 / 2413 

目前進度 :  898 / 2413 

目前進度 :  899 / 2413 

目前進度 :  900 / 2413 

目前進度 :  901 / 2413 

20170520 is holiday!
目前進度 :  902 / 2413 

20170521 is holiday!
目前進度 :  903 / 2413 

目前進度 :  904 / 2413 

目前進度 :  905 / 2413 

目前進度 :  906 / 2413 

目前進度 :  907 / 2413 

目前進度 :  908 / 2413 

20170527 is holiday!
目前進度 :  909 / 2413 

20170528 is holiday!
目前進度 :  910 / 2413 

20170529 is holiday!
目前進度 :  911 / 2413 

20170530 is holiday!
目前進度 :  912 / 2413 

目前進度 :  913 / 2413 

目前進度 :  914 / 2413 

目前進度 :  915 / 2413 

目前進度 :  916 / 2413 

20170604 is holiday!
目前進度 :  917 / 2413 

目前進度 :  918 / 2413 

目前進度 :  919 /

20180217 is holiday!
目前進度 :  1175 / 2413 

20180218 is holiday!
目前進度 :  1176 / 2413 

20180219 is holiday!
目前進度 :  1177 / 2413 

20180220 is holiday!
目前進度 :  1178 / 2413 

目前進度 :  1179 / 2413 

目前進度 :  1180 / 2413 

目前進度 :  1181 / 2413 

20180224 is holiday!
目前進度 :  1182 / 2413 

20180225 is holiday!
目前進度 :  1183 / 2413 

目前進度 :  1184 / 2413 

目前進度 :  1185 / 2413 

20180228 is holiday!
目前進度 :  1186 / 2413 

目前進度 :  1187 / 2413 

目前進度 :  1188 / 2413 

20180303 is holiday!
目前進度 :  1189 / 2413 

20180304 is holiday!
目前進度 :  1190 / 2413 

目前進度 :  1191 / 2413 

目前進度 :  1192 / 2413 

目前進度 :  1193 / 2413 

目前進度 :  1194 / 2413 

目前進度 :  1195 / 2413 

20180310 is holiday!
目前進度 :  1196 / 2413 

20180311 is holiday!
目前進度 :  1197 / 2413 

目前進度 :  1198 / 2413 

目前進度 :  1199 / 2413 

目前進度 :  1200 / 2413 

目前進度 :  1201 / 2413 

目前進度 :  1202 / 2413 

20180317 is holiday!
目前進度 :  1203 / 2413 

20180318 is holiday!
目前進度 :  1204 / 2413 

目前進度 :  1205 / 2413 

目前進度 :  1206 / 2413 

目前進度 :  1207 / 2413 

目

20181201 is holiday!
目前進度 :  1462 / 2413 

20181202 is holiday!
目前進度 :  1463 / 2413 

目前進度 :  1464 / 2413 

目前進度 :  1465 / 2413 

目前進度 :  1466 / 2413 

目前進度 :  1467 / 2413 

目前進度 :  1468 / 2413 

20181208 is holiday!
目前進度 :  1469 / 2413 

20181209 is holiday!
目前進度 :  1470 / 2413 

目前進度 :  1471 / 2413 

目前進度 :  1472 / 2413 

目前進度 :  1473 / 2413 

目前進度 :  1474 / 2413 

目前進度 :  1475 / 2413 

20181215 is holiday!
目前進度 :  1476 / 2413 

20181216 is holiday!
目前進度 :  1477 / 2413 

目前進度 :  1478 / 2413 

目前進度 :  1479 / 2413 

目前進度 :  1480 / 2413 

目前進度 :  1481 / 2413 

目前進度 :  1482 / 2413 

目前進度 :  1483 / 2413 

20181223 is holiday!
目前進度 :  1484 / 2413 

目前進度 :  1485 / 2413 

目前進度 :  1486 / 2413 

目前進度 :  1487 / 2413 

目前進度 :  1488 / 2413 

目前進度 :  1489 / 2413 

20181229 is holiday!
目前進度 :  1490 / 2413 

20181230 is holiday!
目前進度 :  1491 / 2413 

20181231 is holiday!
目前進度 :  1492 / 2413 

20190101 is holiday!
目前進度 :  1493 / 2413 

目前進度 :  1494 / 2413 

目前進度 :  1495 / 2413 

目前進度 :  1496 / 2413 


20190908 is holiday!
目前進度 :  1743 / 2413 

目前進度 :  1744 / 2413 

目前進度 :  1745 / 2413 

目前進度 :  1746 / 2413 

目前進度 :  1747 / 2413 

20190913 is holiday!
目前進度 :  1748 / 2413 

20190914 is holiday!
目前進度 :  1749 / 2413 

20190915 is holiday!
目前進度 :  1750 / 2413 

目前進度 :  1751 / 2413 

目前進度 :  1752 / 2413 

目前進度 :  1753 / 2413 

目前進度 :  1754 / 2413 

目前進度 :  1755 / 2413 

20190921 is holiday!
目前進度 :  1756 / 2413 

20190922 is holiday!
目前進度 :  1757 / 2413 

目前進度 :  1758 / 2413 

目前進度 :  1759 / 2413 

目前進度 :  1760 / 2413 

目前進度 :  1761 / 2413 

目前進度 :  1762 / 2413 

20190928 is holiday!
目前進度 :  1763 / 2413 

20190929 is holiday!
目前進度 :  1764 / 2413 

20190930 is holiday!
目前進度 :  1765 / 2413 

目前進度 :  1766 / 2413 

目前進度 :  1767 / 2413 

目前進度 :  1768 / 2413 

目前進度 :  1769 / 2413 

20191005 is holiday!
目前進度 :  1770 / 2413 

20191006 is holiday!
目前進度 :  1771 / 2413 

目前進度 :  1772 / 2413 

目前進度 :  1773 / 2413 

目前進度 :  1774 / 2413 

20191010 is holiday!
目前進度 :  1775 / 2413 

20191011 is holiday!
目

目前進度 :  2023 / 2413 

目前進度 :  2024 / 2413 

目前進度 :  2025 / 2413 

目前進度 :  2026 / 2413 

目前進度 :  2027 / 2413 

目前進度 :  2028 / 2413 

20200620 is holiday!
目前進度 :  2029 / 2413 

20200621 is holiday!
目前進度 :  2030 / 2413 

目前進度 :  2031 / 2413 

目前進度 :  2032 / 2413 

目前進度 :  2033 / 2413 

20200625 is holiday!
目前進度 :  2034 / 2413 

20200626 is holiday!
目前進度 :  2035 / 2413 

20200627 is holiday!
目前進度 :  2036 / 2413 

20200628 is holiday!
目前進度 :  2037 / 2413 

目前進度 :  2038 / 2413 

目前進度 :  2039 / 2413 

目前進度 :  2040 / 2413 

目前進度 :  2041 / 2413 

目前進度 :  2042 / 2413 

20200704 is holiday!
目前進度 :  2043 / 2413 

20200705 is holiday!
目前進度 :  2044 / 2413 

目前進度 :  2045 / 2413 

目前進度 :  2046 / 2413 

目前進度 :  2047 / 2413 

目前進度 :  2048 / 2413 

目前進度 :  2049 / 2413 

20200711 is holiday!
目前進度 :  2050 / 2413 

20200712 is holiday!
目前進度 :  2051 / 2413 

目前進度 :  2052 / 2413 

目前進度 :  2053 / 2413 

目前進度 :  2054 / 2413 

目前進度 :  2055 / 2413 

目前進度 :  2056 / 2413 

20200718 is holiday!
目前進度 :  2057 / 2413 


目前進度 :  2306 / 2413 

目前進度 :  2307 / 2413 

目前進度 :  2308 / 2413 

20210327 is holiday!
目前進度 :  2309 / 2413 

20210328 is holiday!
目前進度 :  2310 / 2413 

目前進度 :  2311 / 2413 

目前進度 :  2312 / 2413 

目前進度 :  2313 / 2413 

目前進度 :  2314 / 2413 

20210402 is holiday!
目前進度 :  2315 / 2413 

20210403 is holiday!
目前進度 :  2316 / 2413 

20210404 is holiday!
目前進度 :  2317 / 2413 

20210405 is holiday!
目前進度 :  2318 / 2413 

目前進度 :  2319 / 2413 

目前進度 :  2320 / 2413 

目前進度 :  2321 / 2413 

目前進度 :  2322 / 2413 

20210410 is holiday!
目前進度 :  2323 / 2413 

20210411 is holiday!
目前進度 :  2324 / 2413 

目前進度 :  2325 / 2413 

目前進度 :  2326 / 2413 

目前進度 :  2327 / 2413 

目前進度 :  2328 / 2413 

目前進度 :  2329 / 2413 

20210417 is holiday!
目前進度 :  2330 / 2413 

20210418 is holiday!
目前進度 :  2331 / 2413 

目前進度 :  2332 / 2413 

目前進度 :  2333 / 2413 

目前進度 :  2334 / 2413 

目前進度 :  2335 / 2413 

目前進度 :  2336 / 2413 

20210424 is holiday!
目前進度 :  2337 / 2413 

20210425 is holiday!
目前進度 :  2338 / 2413 

目前進度 :  2339 / 2413 



In [ ]:
data.to_csv('data/top3_total_0.csv', sep=',', encoding='utf-8', index=False)
print("Saving Complete!")

Saving Complete!


In [ ]:
# today_date = '20210723'
today_date = datetime.datetime.today().strftime('%Y%m%d')
user_agent = UserAgent(use_cache_server=False)
user_agent = UserAgent()

url = 'https://www.twse.com.tw/fund/T86?response=csv&date='+ today_date + '&selectType=ALLBUT0999'
res = requests.get(url, headers={ 'user-agent': user_agent.random })
text = res.text

cleaned_data = []
for da in text.split('\n'):
    if len(da.split('","')) == 19 and da.split('","')[0][0] != '=':
        cleaned_data.append([ele.replace('",\r','').replace('"','') 
                             for ele in da.split('","')])
df = pd.DataFrame(cleaned_data, columns = cleaned_data[0])[1:]
df.columns = ['code', 'name', 'FI_BUY', 'FI_SELL', 'FI_TOTAL', 'FI_SELF_BUY', 'FI_SELF_SELL', 'FI_SELF_TOTAL', 'SIT_BUY', 'SIT_SELL', 'SIT_TOTAL', 'DL_TOTAL', 'DL_SELF_BUY', 'DL_SELF_SELL', 'DL_SELF_TOTAL', 'DL_HEDGE_BUY', 'DL_HEDGE_SELL', 'DL_HEDGE_TOTAL', 'TOP3_TOTAL']
df['date'] = today_date
df = df[['date', 'code', 'name', 'FI_BUY', 'FI_SELL', 'FI_TOTAL', 'FI_SELF_BUY', 'FI_SELF_SELL', 'FI_SELF_TOTAL', 'SIT_BUY', 'SIT_SELL', 'SIT_TOTAL', 'DL_TOTAL', 'DL_SELF_BUY', 'DL_SELF_SELL', 'DL_SELF_TOTAL', 'DL_HEDGE_BUY', 'DL_HEDGE_SELL', 'DL_HEDGE_TOTAL', 'TOP3_TOTAL']]
df = df[df['code'].apply(lambda x: len(x) < 5)].reset_index(drop=True)
df.to_csv('data/top3_total_' + today_date + '.csv', sep=',', encoding='utf-8', index=False)
print("Saving Complete!")

Saving Complete!
